In [46]:
import pandas as pd
from datetime import datetime
import dateutil.relativedelta
from re import sub
# import pypyodbc as odbc

In [2]:
pd.options.mode.copy_on_write = True

# Test database

In [ ]:
# drive_name = 'SQL SERVER'
# server_name = 'DESKTOP-29355SM\SQLEXPRESS'
# database_name = 'Bank'

# connection_string = f"""
#     DRIVER={{{drive_name}}};
#     SERVER={server_name};
#     DATABASE={database_name};
#     Trust_Connection=yes;
# """

# conn = odbc.connect(connection_string)

In [ ]:
# cursor = conn.cursor()

In [ ]:
# cursor.execute("""
# Create table Persons (
#     PersonID int,
#     LastName varchar(255),
#     FirstName varchar(255),
#     Address varchar(255),
#     City varchar(255)
# )
# """)

In [ ]:
# df = pd.read_sql('Select * from account', conn)

In [ ]:
# conn.commit()

# Function

In [3]:
def extract_date_review(text_date, crawl_date): # Biến đổi khoảng thời gian (1 tháng trước, 2 ngày trước) thằnh ngày cụ thể và trả về datetime
    sub_text = text_date.split(' ')
    date_cmt = ''

    if 'năm' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(t))
    elif 'tháng' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(months=int(t))
    elif 'tuần' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(weeks==int(t))
    elif 'ngày' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(days=int(t))
    else:
        date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date()

    return date_cmt

In [4]:
def create_id(last_id): # Tạo ID mới và trả về string
    last_id_list = []
    newest_id = ''

    for s in list(last_id):
        if s.isnumeric():
            last_id_list.append(s)
        else:
            newest_id += s
    
    last_id = str(int(''.join(last_id_list)) + 1)
    if len(last_id_list) != len(last_id):
        num_0 = len(last_id_list) - len(last_id)
        newest_id += '0'*num_0 + last_id
    else:
        newest_id += last_id
    
    return newest_id

In [5]:
def add_data(file_table_name, df, id_format=None): # Thêm df vào file save(df không chứa cột ID, thứ tự cột giống với file save và nếu file save trống thì cần đưa ID format (S000, PL00,...))
    table = pd.read_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name))

    newest_id = ''
    if id_format != None:
        newest_id = create_id(id_format)
    else:
        newest_id = create_id(table.iloc[-1][0])

    list_id = []
    for i in range(len(df)):
        list_id.append(newest_id)
        newest_id = create_id(newest_id)
    df.insert(loc=0, column='ID', value=list_id)

    df.columns = table.columns
    table = pd.concat([table, df], ignore_index=True)
    table.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    table.to_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name), index=False)

In [6]:
def add_shop(new_shop_df): # Thêm shop mới vào file Shop.csv (new_shop_df chỉ bao gồm 'Shop name', 'Shop rating' và đặt đúng thứ tự)
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    check_shop_list = new_shop_df.drop_duplicates(subset=new_shop_df.columns[0], keep='last', ignore_index=True)
    drop_index = []

    for index_shop in range(len(check_shop_list)):
        if check_shop_list.iloc[index_shop][0] in shop_df['ShopName'].values:
            drop_index.append(index_shop)

    check_shop_list.drop(drop_index, inplace=True)

    if len(shop_df) == 0:
        add_data('Shop.csv', check_shop_list, id_format='S000')
    else:
        add_data('Shop.csv', check_shop_list)

# Main

In [7]:
platform_df = pd.read_csv(r'..\Data\Preprocessed_data\Platform.csv')
platform_df

,PlatformID,PlatformName
0,PL01,Tiki
1,PL02,Lazada
2,PL03,Shopee
3,PL04,Youtube
4,PL05,Tinhte


In [8]:
product_df = pd.read_csv(r'..\Data\Preprocessed_data\Product.csv')
product_df

,ProductID,ProductName,Brand
0,P001,iPhone 15 Pro Max,iPhone
1,P002,Galaxy S23 Ultra,Samsung


# Tiki

#### Product info

In [ ]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()

    tiki_product_info = pd.read_csv(r'..\Data\Tiki\%s_info.csv'%('_'.join(product_name.split(' '))))

    # Loại bỏ dấu "." trong cột "Price" và đổi định dạng thành int
    tiki_product_info['Price'] = tiki_product_info['Price'].str.replace('.', '', regex=True).astype(int)

    # Lọc lấy data của ngày crawl mới nhất nếu sp không có sự thay đổi giá
    tiki_product_info.drop_duplicates(subset=['Name', 'Price'], keep='last', inplace=True, ignore_index=True)

    # Thêm cột ID của sp
    tiki_product_info['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    tiki_product_info['PlatformID'] = 'PL01'

    # Thêm shop mới vào file Shop.csv nếu có
    shop_list = tiki_product_info[['Shop name', 'Shop rating']].copy()
    add_shop(shop_list)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        tiki_product_info.loc[tiki_product_info['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    add_df = tiki_product_info[['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Crawl date']]

    # Lưu vào file save Market.csv
    market_df = pd.read_csv(r'..\Data\Preprocessed_data\Market.csv')
    if len(market_df) == 0:
        add_data('Market.csv', add_df, 'MK000')
    else:
        add_data('Market.csv', add_df)

#### Reviews

In [193]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()

    tiki_reviews = pd.read_csv(r'..\Data\Tiki\%s_reviews.csv'%('_'.join(product_name.split(' '))))

    # Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
    actual_date_list = []

    for num in range(len(tiki_reviews)):
        text_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][0]
        crawl_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][1]

        actual_date = extract_date_review(text_date, crawl_date)
        actual_date_list.append(actual_date)

    tiki_reviews['Rating date'] = actual_date_list
    tiki_reviews.drop(columns=['Crawl date'], inplace=True)

    # Biến đổi cột "Rating" thành thang đo likert
    rating_likert = {'Rất không hài lòng': 1, 'Không hài lòng': 2, 'Bình thường': 3, 'Hài lòng': 4, 'Cực kì hài lòng': 5}

    for num in range(len(tiki_reviews)):
        rating = tiki_reviews.loc[num, 'Rating']
        for k, v in rating_likert.items():
            if rating == k:
                tiki_reviews.loc[num, 'Rating'] = v

    # Loại bỏ những dòng trùng "Reviewer name" và "Content"
    tiki_reviews.drop_duplicates(subset=['Reviewer name', 'Content'], keep='last', inplace=True, ignore_index=True)

    # Thêm cột ID của sp
    tiki_reviews['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    tiki_reviews['PlatformID'] = 'PL01'

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        tiki_reviews.loc[tiki_reviews['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    add_df = tiki_reviews[['Content', 'ProductID', 'PlatformID', 'Rating date', 'ShopID', 'Rating']]

    # Lưu vào file save Market.csv
    ecom_reviews_df = pd.read_csv(r'..\Data\Preprocessed_data\EcomReviews.csv')
    if len(ecom_reviews_df) == 0:
        add_data('EcomReviews.csv', add_df, 'ER00000000')
    else:
        add_data('EcomReviews.csv', add_df)

# Lazada

In [68]:
product_name = 'Galaxy S23 Ultra'.lower()

#### Product info

In [89]:
lazada_product_info = pd.read_csv(r'..\Data\Lazada\%s_info.csv'%('_'.join(product_name.split(' '))))

lazada_product_info = lazada_product_info[['Name', 'Brand', 'Quantity', 'Reviews count',
                                            'Star rating', 'Price', 'Crawl date', 'Shop name',
                                            'Shop rating']]

# Chỉnh sửa cột "Quantity" thành số
if 'int' not in str(lazada_product_info['Quantity'].dtypes):
    for index in range(len(lazada_product_info)):
        count = lazada_product_info.loc[index, 'Quantity']
        sold_count = ''

        if not count.isnumeric():
            sold_count = sub('\D', '', count)
            if len(sold_count) == 1:
                sold_count = int(sold_count) * 1000
        else:
            sold_count = int(count)
        
        lazada_product_info.loc[index, 'Quantity'] = sold_count

# Chỉnh sửa cột "Quantity" thành số
for index in range(len(lazada_product_info)):
    review_count = lazada_product_info.loc[index, 'Reviews count']
    if review_count == 'Không có đánh giá':
        lazada_product_info.loc[index, 'Reviews count'] = 0
    else:
        lazada_product_info.loc[index, 'Reviews count'] = int(review_count.replace('đánh giá', ''))

# Chỉnh sửa cột "Shop rating" về thang đo 5
for index in range(len(lazada_product_info)):
    shop_rating = lazada_product_info.loc[index, 'Shop rating']

    if shop_rating == 'Không có đánh giá':
        lazada_product_info.loc[index, 'Shop rating'] = ''
    else:
        rating = int(sub('\D', '', shop_rating))
        lazada_product_info.loc[index, 'Shop rating'] = round((5*rating)/100, 1)

In [91]:
lazada_product_info

,Name,Brand,Quantity,Reviews count,Star rating,Price,Crawl date,Shop name,Shop rating
0,[TẾT SALE TO CHỈ 15.01] Điện thoại Samsung Gal...,Samsung,674,140,4.9,19100000,2024-01-15,SAMSUNG Flagship Store,4.5
1,Điện thoại Samsung Galaxy S23 Ultra 8GB/256GB ...,Samsung,110,20,4.9,25990200,2024-01-15,Samsung Store,4.8
2,Điện Thoại Samsung Galaxy S23 Ultra | 256GB - ...,Samsung,13,1,1.0,26380000,2024-01-15,SAMSUNG PREMIUM STORE,4.7
3,[Hàng chính hãng] Samsung Galaxy S23 Ultra 5G ...,Samsung,165,4,4.0,22890000,2024-01-15,FPT SHOP,4.8
4,Điện thoại Samsung Galaxy S23 Ultra - Mầu độc ...,Samsung,0,0,0.0,22120000,2024-01-15,SAMSUNG PREMIUM STORE,4.7
5,Điện Thoại Samsung Galaxy S23 Ultra 5G 256GB,Samsung,0,0,0.0,21090000,2024-01-15,Tablet Plaza Store,
6,Điện thoại Samsung Galaxy S23 Ultra,Samsung,198,43,4.8,22890000,2024-01-15,NGUYỄN KIM,5.0
7,Điện Thoại Samsung Galaxy S23 Ultra 512GB Chín...,Samsung,0,0,0.0,28900000,2024-01-15,Nam Á Store - SPS,5.0
8,Điện Thoại Samsung Galaxy S23 Ultra 5G - Hàng ...,Samsung,0,0,0.0,30000000,2024-01-15,GDV GIA DỤNG VIỆT,5.0
9,Điện thoại Samsung Galaxy S23 Ultra- Hàng Chín...,Samsung,246,71,4.9,22800000,2024-01-15,HAPPY PHONE,4.9


In [ ]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()

    lazada_product_info = pd.read_csv(r'..\Data\Tiki\%s_info.csv'%('_'.join(product_name.split(' '))))

    # Loại bỏ dấu "." trong cột "Price" và đổi định dạng thành int
    lazada_product_info['Price'] = lazada_product_info['Price'].str.replace('.', '', regex=True).astype(int)

    # Lọc lấy data của ngày crawl mới nhất nếu sp không có sự thay đổi giá
    lazada_product_info.drop_duplicates(subset=['Name', 'Price'], keep='last', inplace=True, ignore_index=True)

    # Thêm cột ID của sp
    lazada_product_info['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    lazada_product_info['PlatformID'] = 'PL01'

    # Thêm shop mới vào file Shop.csv nếu có
    shop_list = lazada_product_info[['Shop name', 'Shop rating']].copy()
    add_shop(shop_list)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        lazada_product_info.loc[lazada_product_info['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    add_df = lazada_product_info[['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Crawl date']]
    print(add_df)
    # # Lưu vào file save Market.csv
    # market_df = pd.read_csv(r'..\Data\Preprocessed_data\Market.csv')
    # if len(market_df) == 0:
    #     add_data('Market.csv', add_df, 'MK000')
    # else:
    #     add_data('Market.csv', add_df)

#### Reviews

# Websites

#### Tinhte

In [ ]:
tinhte_content = pd.read_csv(r'..\Data\TinhTe\TinhTe_Iphone15pm_InfoPosts2.csv')

# Youtube